In [1]:
#!unzip /content/archive.zip

In [2]:
!pip install tensorflow

1️⃣ استيراد المكتبات

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


2️⃣ بناء النموذج (4 فئات)

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x= Dense(256, activation='relu')(x)
x= Dense(128, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # ← 4 فئات

model = Model(inputs=base_model.input, outputs=predictions)

# تثبيت طبقات ResNet50
for layer in base_model.layers:
    layer.trainable = False

# تجميع النموذج
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


3️⃣ تحضير بيانات التدريب والاختبار

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r'C:\Users\HP\Desktop\Neuer Ordner\projeckt\archive\Training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    r'C:\Users\HP\Desktop\Neuer Ordner\projeckt\archive\Testing',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


4️⃣ تدريب النموذج

In [6]:
model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=100
)


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 293s 2s/step - accuracy: 0.4139 - loss: 1.2557 - val_accuracy: 0.4455 - val_loss: 1.0899
Epoch 2/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.6121 - loss: 0.9413 - val_accuracy: 0.5736 - val_loss: 0.9956
Epoch 3/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.6618 - loss: 0.8427 - val_accuracy: 0.6957 - val_loss: 0.7925
Epoch 4/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.6805 - loss: 0.7936 - val_accuracy: 0.6583 - val_loss: 0.7906
Epoch 5/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.7151 - loss: 0.7156 - val_accuracy: 0.6568 - val_loss: 0.7847
Epoch 6/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.7040 - loss: 0.7233 - val_accuracy: 0.5721 - val_loss: 0.9055
Epoch 7/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.7072 - loss: 0.7030 - val_accuracy: 0.6796 - val_loss: 0.7212
Epoch 8/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.7105 - loss: 0.7009 - 

5️⃣ حفظ النموذج

In [7]:
model.save('brain_tumor_classifier_22222.h5')


In [8]:
!pip install gradio


6️⃣ تحميل صورة والتنبؤ بها

In [20]:
# ====== Imports ======
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
from pathlib import Path
import os

# ====== Load trained model ======
model = load_model('brain_tumor_classifier_22222.h5')

# ====== Class names ======
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# ====== Directory to save results ======
results_dir = Path("Brain_Tumor_Results")
results_dir.mkdir(parents=True, exist_ok=True)

# ====== Prediction and report generation function ======
def predict_and_generate_report(img):
    # Preprocess image
    resized_img = img.resize((224, 224))
    img_array = image.img_to_array(resized_img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction[0])
    predicted_class = class_names[predicted_class_index]

    # Diagnosis result
    if predicted_class == 'notumor':
        label = "No Tumor Detected (Benign)"
        result_type = "Benign"
        color = 'green'
    else:
        label = f"Possible Tumor Detected ({predicted_class})"
        result_type = "Possibly Malignant"
        color = 'red'

    # Report details
    model_name = "ResNet50"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    file_stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    pdf_filename = results_dir / f"diagnosis_{file_stamp}.pdf"

    # Generate PDF
    with PdfPages(pdf_filename) as pdf:
        # Page 1
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.text(10, 20, label, fontsize=14, color='white',
                 bbox=dict(facecolor=color, alpha=0.7, boxstyle='round,pad=0.4'))
        info_text = f"Model: {model_name} | {timestamp}"
        plt.text(10, 210, info_text, fontsize=10, color='white',
                 bbox=dict(facecolor='gray', alpha=0.5, boxstyle='round,pad=0.3'))
        pdf.savefig(bbox_inches='tight')
        plt.close()

        # Page 2
        plt.figure(figsize=(6, 4))
        plt.axis('off')
        report_text = (
            f"Date: {timestamp}\n"
            f"Model: {model_name}\n"
            f"Predicted Class: {predicted_class}\n"
            f"Final Diagnosis: {result_type}\n"
        )
        plt.text(0, 1, report_text, fontsize=12, va='top')
        pdf.savefig(bbox_inches='tight')
        plt.close()

    return label, str(pdf_filename)

# ====== Gradio Interface ======
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Brain Tumor Diagnosis Using ResNet50")
    gr.Markdown("Upload an MRI image to get a diagnosis and download the report as a PDF.")

    with gr.Row():
        image_input = gr.Image(type="pil", label="📤 Upload MRI Image")
        result_output = gr.Textbox(label="🔍 Diagnosis Result")
    
    download_file = gr.File(label="📄 Download Report", interactive=True)

    analyze_button = gr.Button("Analyze Image")

    # On button click
    def on_click(img):
        result, pdf_path = predict_and_generate_report(img)
        return result, pdf_path

    analyze_button.click(
        fn=on_click,
        inputs=[image_input],
        outputs=[result_output, download_file]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
